In [ ]:
# Agent Framework Comparison: SPADE vs JADE
# Analysis of multi-agent negotiation logs

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import numpy as np
from io import StringIO

# Set up plotting style
plt.style.use('default')
sns.set_palette("husl")

## Data Loading and Preprocessing

spadee_data = 
# Load data into DataFrames
spade_df = pd.read_csv(StringIO(spade_data))
jade_df = pd.read_csv(StringIO(jade_data))

# Add framework identifier
spade_df['framework'] = 'SPADE'
jade_df['framework'] = 'JADE'

print("SPADE Framework Data:")
print(f"Shape: {spade_df.shape}")
print(spade_df.head())
print("\nJADE Framework Data:")
print(f"Shape: {jade_df.shape}")
print(jade_df.head())

## Timestamp Analysis

# Convert timestamps to datetime
spade_df['timestamp'] = pd.to_datetime(spade_df['timestamp'])
jade_df['timestamp'] = pd.to_datetime(jade_df['timestamp'])

# Calculate time differences
spade_duration = (spade_df['timestamp'].max() - spade_df['timestamp'].min()).total_seconds()
jade_duration = (jade_df['timestamp'].max() - jade_df['timestamp'].min()).total_seconds()

print(f"\n=== TIMING ANALYSIS ===")
print(f"SPADE total duration: {spade_duration:.3f} seconds")
print(f"JADE total duration: {jade_duration:.3f} seconds")
print(f"SPADE date: {spade_df['timestamp'].iloc[0].date()}")
print(f"JADE date: {jade_df['timestamp'].iloc[0].date()}")

## Agent Communication Patterns

def analyze_communication_patterns(df, framework_name):
    print(f"\n=== {framework_name} COMMUNICATION PATTERNS ===")
    
    # Unique agents
    senders = set(df['sender'].unique())
    receivers = set(df['receivers'].unique())
    all_agents = senders.union(receivers)
    
    print(f"Total unique agents: {len(all_agents)}")
    print(f"Agents: {sorted(all_agents)}")
    
    # Message types
    actions = df['agentAction'].value_counts()
    performatives = df['performative'].value_counts()
    
    print(f"\nAgent Actions:")
    for action, count in actions.items():
        print(f"  {action}: {count}")
    
    print(f"\nPerformatives:")
    for perf, count in performatives.items():
        print(f"  {perf}: {count}")
    
    # Conversation analysis
    conversations = df['conversationId'].nunique()
    print(f"\nUnique conversations: {conversations}")
    
    return {
        'agents': all_agents,
        'actions': actions,
        'performatives': performatives,
        'conversations': conversations
    }

spade_analysis = analyze_communication_patterns(spade_df, "SPADE")
jade_analysis = analyze_communication_patterns(jade_df, "JADE")

## Message Flow and Sequence Analysis

fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 1. Message frequency over time
axes[0, 0].plot(spade_df['sequenceId'], spade_df.index, 'o-', label='SPADE', alpha=0.7)
axes[0, 0].plot(jade_df['sequenceId'], jade_df.index, 's-', label='JADE', alpha=0.7)
axes[0, 0].set_xlabel('Sequence ID')
axes[0, 0].set_ylabel('Message Index')
axes[0, 0].set_title('Message Sequence Flow')
axes[0, 0].legend()
axes[0, 0].grid(True)

# 2. Action types comparison
combined_actions = pd.concat([
    spade_analysis['actions'].to_frame('count').assign(framework='SPADE'),
    jade_analysis['actions'].to_frame('count').assign(framework='JADE')
]).reset_index()

action_pivot = combined_actions.pivot(index='index', columns='framework', values='count').fillna(0)
action_pivot.plot(kind='bar', ax=axes[0, 1])
axes[0, 1].set_title('Agent Actions Comparison')
axes[0, 1].set_xlabel('Action Type')
axes[0, 1].set_ylabel('Count')
axes[0, 1].tick_params(axis='x', rotation=45)

# 3. Performatives comparison
combined_perf = pd.concat([
    spade_analysis['performatives'].to_frame('count').assign(framework='SPADE'),
    jade_analysis['performatives'].to_frame('count').assign(framework='JADE')
]).reset_index()

perf_pivot = combined_perf.pivot(index='index', columns='framework', values='count').fillna(0)
perf_pivot.plot(kind='bar', ax=axes[1, 0])
axes[1, 0].set_title('Performatives Comparison')
axes[1, 0].set_xlabel('Performative Type')
axes[1, 0].set_ylabel('Count')
axes[1, 0].tick_params(axis='x', rotation=45)

# 4. Agent participation
spade_agent_counts = pd.concat([spade_df['sender'], spade_df['receivers']]).value_counts()
jade_agent_counts = pd.concat([jade_df['sender'], jade_df['receivers']]).value_counts()

axes[1, 1].bar(range(len(spade_agent_counts)), spade_agent_counts.values, 
               alpha=0.7, label='SPADE', width=0.4)
axes[1, 1].bar([i+0.4 for i in range(len(jade_agent_counts))], jade_agent_counts.values, 
               alpha=0.7, label='JADE', width=0.4)
axes[1, 1].set_title('Agent Message Participation')
axes[1, 1].set_xlabel('Agents (ordered by frequency)')
axes[1, 1].set_ylabel('Message Count')
axes[1, 1].legend()

plt.tight_layout()
plt.show()

## Content Analysis

def analyze_content(df, framework_name):
    print(f"\n=== {framework_name} CONTENT ANALYSIS ===")
    
    # Content structure analysis
    content_lengths = df['content'].str.len()
    print(f"Average content length: {content_lengths.mean():.1f} characters")
    print(f"Content length range: {content_lengths.min()} - {content_lengths.max()}")
    
    # Sample content
    print(f"\nSample content entries:")
    for i, content in enumerate(df['content'].head(3)):
        print(f"  {i+1}: {content[:100]}...")
    
    return content_lengths

spade_content = analyze_content(spade_df, "SPADE")
jade_content = analyze_content(jade_df, "JADE")

## Conversation ID Analysis

def analyze_conversations(df, framework_name):
    print(f"\n=== {framework_name} CONVERSATION ANALYSIS ===")
    
    conv_stats = df.groupby('conversationId').agg({
        'sequenceId': ['count', 'min', 'max'],
        'performative': lambda x: x.nunique(),
        'agent': lambda x: x.nunique()
    }).round(2)
    
    conv_stats.columns = ['msg_count', 'first_seq', 'last_seq', 'perf_types', 'agent_count']
    print(conv_stats)
    
    return conv_stats

spade_conv = analyze_conversations(spade_df, "SPADE")
jade_conv = analyze_conversations(jade_df, "JADE")

## Key Differences Summary

print(f"\n" + "="*60)
print("KEY DIFFERENCES BETWEEN SPADE AND JADE")
print("="*60)

print(f"\n1. ADDRESSING SCHEME:")
print(f"   SPADE: Uses email-like addresses (agent@localhost)")
print(f"   JADE:  Uses simple agent names (SalaCM3, SalaIC3)")

print(f"\n2. MESSAGE FLOW:")
print(f"   SPADE: {len(spade_df)} messages over {spade_duration:.3f} seconds") 
print(f"   JADE:  {len(jade_df)} messages over {jade_duration:.3f} seconds")

print(f"\n3. CONVERSATION IDs:")
print(f"   SPADE: Shorter IDs (e.g., 'neg-ROJAS RODRIGUEZ RO-12')")
print(f"   JADE:  Longer IDs with timestamps (e.g., 'neg-ROJAS RODRIGUEZ RO-SalaCM3-1748490278054')")

print(f"\n4. CONTENT FORMAT:")
print(f"   SPADE: JSON-like structured data")
print(f"   JADE:  Comma-separated values")

print(f"\n5. AGENT INTERACTION:")
print(f"   SPADE: {len(spade_analysis['agents'])} unique agents")
print(f"   JADE:  {len(jade_analysis['agents'])} unique agents")

print(f"\n6. PERFORMATIVES:")
spade_perfs = set(spade_df['performative'].unique())
jade_perfs = set(jade_df['performative'].unique())
print(f"   SPADE: {spade_perfs}")
print(f"   JADE:  {jade_perfs}")
print(f"   Common: {spade_perfs.intersection(jade_perfs)}")
print(f"   SPADE only: {spade_perfs - jade_perfs}")
print(f"   JADE only: {jade_perfs - spade_perfs}")

## Framework Comparison Table

comparison_data = {
    'Metric': [
        'Total Messages',
        'Duration (seconds)',
        'Unique Agents',
        'Conversations',
        'Avg Content Length',
        'SEND Actions',
        'RECEIVE Actions',
        'CFP Performatives',
        'Addressing Style'
    ],
    'SPADE': [
        len(spade_df),
        f"{spade_duration:.3f}",
        len(spade_analysis['agents']),
        spade_analysis['conversations'],
        f"{spade_content.mean():.1f}",
        spade_analysis['actions'].get('SEND', 0),
        spade_analysis['actions'].get('RECEIVE', 0),
        spade_analysis['performatives'].get('cfp', 0),
        'Email-like'
    ],
    'JADE': [
        len(jade_df),
        f"{jade_duration:.3f}",
        len(jade_analysis['agents']),
        jade_analysis['conversations'],
        f"{jade_content.mean():.1f}",
        jade_analysis['actions'].get('SEND', 0),
        jade_analysis['actions'].get('RECEIVE', 0),
        jade_analysis['performatives'].get('CFP', 0),
        'Simple names'
    ]
}

comparison_df = pd.DataFrame(comparison_data)
print(f"\n{comparison_df.to_string(index=False)}")

print(f"\n" + "="*60)
print("ANALYSIS COMPLETE")
print("="*60)